In [1]:
import numpy as np
from collections import namedtuple
from fractions import Fraction
from scipy.optimize import linprog
import matplotlib.pyplot as plt
import seaborn as sns

Item = namedtuple('Item', ('value', 'weight', 'ratio'))

In [2]:
# Fast, but Approximate result
# def knapsack_greedy(values, weights, W, n):
#     total_value = 0
#     values, weights = zip(*sorted(zip(values, weights), key=lambda t: t[0]/t[1], reverse=True))
#     for value, weight in zip(values, weights):
#         if W - weight > 0:
#             total_value += value
#             W -= weight
#     return total_value

## Implementation of offline knapsack and online knapsack

In [3]:
# using linear programming
def offline_knapsack(items, max_weight):
    # construct the objective function: maximize the sum of the values of the items
    objective = np.array([-item.value for item in items])
    
    # construct the constraints:
    # - the sum of the weights of the items must be less than or equal to the maximum weight
    # - all weights and values must be non-negative
    constraints = np.array([[item.weight for item in items]])
    b_ub = np.array([max_weight])
    bounds = bounds = [(0, 1),] * len(items)

    
    result = linprog(
        c=objective,
        A_ub=constraints,
        b_ub=b_ub,
        bounds=bounds,
        method="HiGHS",
    )
    
    # return the maximum value that can be obtained
    return result.fun


# Implementation of the online threshold algorithm in the slides
def online_knapsack(items, p_min, p_max, max_weight):
    total_value = 0
    y = 0
    beta = max_weight / (1 + np.log(p_max / p_min))
    # print(f"beta: {beta}")
    for item in items:
        threshold = p_min if y < beta else p_min * np.exp(y/beta-1)
        # print(threshold)
        if item.ratio > threshold and y + item.weight <= max_weight:
            y += item.weight
            total_value += item.value
        
    return total_value

In [4]:
def test(n=5000, epsilon=0.05, max_weight=1, p_min=1, p_max=10, adverserial=False):
    
    offline_results = []
    online_results = []
    for i in range(100):

        weights = np.random.uniform(0, epsilon, n)
        values = np.random.uniform(p_min, p_max, n) * weights
        items = []
        for value, weight in zip(values, weights):
            item = Item(value, weight, value/weight)
            items.append(item)
    
        if adverserial:
            items = sorted(items, key=lambda item: item.ratio)
    
        offline = -offline_knapsack(items, max_weight)
        online = online_knapsack(items, p_min, p_max, max_weight)
    
        offline_results.append(offline)
        online_results.append(online)
    
    return np.mean(offline_results), np.mean(online_results)

## Infinitesimal settings

Assumption 1: each item’s weight is very small $w_t$ → 0.<br />
Assumption 2: the value density of each item is bounded: $$\forall t \quad \rho_{min} \leq {v_t \over w_t} \leq \rho_{max}$$

$\rho_{min}, \rho_{max}$ should not be too big or too small

$\alpha = 1 + np.log({p_{max} \over p_{min}})$<br />
$\beta = {1 \over \alpha}$

In [5]:
p_min = 1
p_max = 5

alpha = 1 + np.log(p_max / p_min)
print(f"alpha: {alpha}")
print(f"beta:  {1 / alpha}")

n = 2500
max_weight = 1
epsilon = 0.02

alpha: 2.6094379124341005
beta:  0.383224293337255


In [6]:
offline_avg, online_avg = test(n=n, epsilon=epsilon, max_weight=max_weight, p_min=p_min, p_max=p_max, adverserial=False)
print(f"Opt / Alg: {offline_avg / online_avg} | offline_average: {offline_avg} | online_average: {online_avg}")

Opt / Alg: 1.4283166101103393 | offline_average: 4.920760600771931 | online_average: 3.4451469414696483


In [7]:
offline_avg, online_avg = test(n=n, epsilon=epsilon, max_weight=max_weight, p_min=p_min, p_max=p_max, adverserial=True)
print(f"Opt / Alg: {offline_avg / online_avg} | offline_average: {offline_avg} | online_average: {online_avg}")

Opt / Alg: 2.5888289674406093 | offline_average: 4.919832462791187 | online_average: 1.9004084567452422


## When the weight of each item is not infinitesimal

In [8]:
p_min = 1
p_max = 5

alpha = 1 + np.log(p_max / p_min)
print(f"alpha: {alpha}")
print(f"beta:  {1 / alpha}")

n = 2500
max_weight = 100
epsilon = 20

alpha: 2.6094379124341005
beta:  0.383224293337255


In [9]:
offline_avg, online_avg = test(n=n, epsilon=epsilon, max_weight=max_weight, p_min=p_min, p_max=p_max, adverserial=False)
print(f"Opt / Alg: {offline_avg / online_avg} | offline_average: {offline_avg} | online_average: {online_avg}")

Opt / Alg: 1.4978429210751478 | offline_average: 499.10943537042374 | online_average: 333.2188097615498


In [10]:
offline_avg, online_avg = test(n=n, epsilon=epsilon, max_weight=max_weight, p_min=p_min, p_max=p_max, adverserial=True)
print(f"Opt / Alg: {offline_avg / online_avg} | offline_average: {offline_avg} | online_average: {online_avg}")

Opt / Alg: 2.950358308583454 | offline_average: 499.1377983667121 | online_average: 169.17870514729498
